In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Importacion de librerias

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import argparse
import tensorflow as tf
from IPython.display import display, clear_output
from random import randint

# Configuraciones de TensorFlow

In [3]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

### Modo eagerly

In [4]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

False

# Carga de Datos

In [5]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

'/home/jefelitman/DataSets/ucf101/split_1'

In [6]:
batch_size = 32
size = [112,112]
frames = 16
canales = 3

In [7]:
import sys
sys.path.append('../')

from Datasets_utils.DatasetsLoader import VideoDataGenerator

dataset = VideoDataGenerator(root_path,batch_size,True)

# Red Neuronal

In [9]:
from LTC import LTC

Sesion = tf.Session(config=config)

video_shape = [frames]+size+[canales]

ltc = LTC(101,batch_size,video_shape)

ltc.enable_training()

W1015 03:06:31.430803 140394066609984 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:15: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [10]:
ltc.inicializar_modelo(0.003,0.5)
init = tf.global_variables_initializer()
Sesion.run(init)

W1015 03:06:40.030470 140394066609984 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:112: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1015 03:06:40.031846 140394066609984 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:113: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1015 03:06:40.033422 140394066609984 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1015 03:06:40.084724 140394066609984 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC.py:157: The name tf.nn.xw_plus_b is deprecated. Please use tf.compat.v1.nn.xw_plus_b instea

In [11]:
#Como es primera ver corriendo este modelo entonces no debo cargar nada
saver = tf.train.Saver()
steps = 160000
epoch = 0
for paso in range(steps):
    clear_output(wait=True)
    
    #Primero voy salvando el modelo de 40000 en 40000
    if paso % 40000 == 0:
        saved_path = saver.save(Sesion, '../Checkpoints/LTC_112x112x16_rgb')
        print('Modelo salvado en: {}'.format(saved_path))
        
    if paso == 80000 or paso == 125000:
        ltc.lr = ltc.lr * 0.1
    
    if dataset.train_batch_index == 0:
        epoch += 1
        
    print("Epoch: ",epoch)
    print("Train batch: ",dataset.train_batch_index+1,"/",dataset.train_batches)
    
    train_batch, train_labels = dataset.get_next_train_batch(size,frames,canales)
    
    #Propago
    Sesion.run(ltc.entrenar,feed_dict={ltc.x: train_batch, ltc.y: train_labels})
    
    perdida = Sesion.run(ltc.perdida,feed_dict={ltc.x: train_batch, ltc.y: train_labels})
    precision = Sesion.run(ltc.precision,feed_dict={ltc.x: train_batch, ltc.y: train_labels})
    print("Loss: ",perdida," Acuraccy: ",precision)
    
print("Final de entrenamiento")

Epoch:  1
Train batch:  262 / 299


KeyboardInterrupt: 

In [ ]:
Sesion.close()
del(Sesion)